In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import json

### Part 1
Scraping School map data from MNPS site (https://mnps.maps.arcgis.com/home/item.html?id=a25c1ef058a247cfa2636a396f3aaedd#data). Table has information for geographic location (latitude and longitude) for each MNPS school. This information will be used for map generation. 

In [ ]:
URL = 'https://services.arcgis.com/GBzOxI4K7WHYqcYy/arcgis/rest/services/Schools_Map/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=1000&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D'

In [ ]:
response = requests.get(URL)

In [ ]:
School_map_data = pd.DataFrame([x['attributes'] for x in response.json()['features']])

In [ ]:
School_map_data

In [ ]:
School_map_data.info()

In [ ]:
School_map_data = School_map_data.drop(columns = {'OBJECTID', 'SHORT_NAME', 'GR_TIER', 'NOTES','SCHBRD_DIST','SCHBRD_MBR', 'OPTIONSCH_CURR', 'OPTIONSCH_NEXT', 'OPTID_CURR', 'OPTID_NEXT', 'TN_HOUSE', 'TN_SENATE'})
School_map_data = School_map_data.rename(columns = {'CLUS_NAME':'CLUSTER_NAME','LONG_':'LONGITUDE', 'LAT_':'LATITUDE'})
School_map_data.info()

In [ ]:
School_map_data["SCHOOL_NAME"].unique()

In [ ]:
School_map_data['ADDRESS'] = School_map_data.SHORT_ADDR.str.cat(School_map_data.CITY, sep=', ')
School_map_data['ADDRESS'] = School_map_data['ADDRESS'].astype(str) + ', TN-'
School_map_data['ADDRESS'] = School_map_data.ADDRESS.str.cat(School_map_data.ZIP, sep='')
School_map_data

In [ ]:
enrollment = pd.read_csv('Data/enrollment.csv')
enrollment

In [ ]:
enrollment = enrollment.rename(columns = {'school': 'SCHOOL_NAME', 'sl':'SCHOOL_TYPE', 'enrollment' : 'TOTAL_ENROLLMENT', 'years' : 'YEARS'})
enrollment

In [ ]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].replace({'Antioch High School': 'Antioch High' , 'Whites Creek High School': 'Whites Creek High', Bordeaux Enhanced Option School})
enrollment
'Brick Church Middle School'
Brookmeade Elementary School
Cameron Middle School'
'Cora Howe School'
'Dalewood Middle School
'Early College High School'
'East Nashville School', 'Ewing Park Middle School'
'Glencliff High School',
'H.G. Hill Middle School'
 'Martha Vaught Middle School', 'Martin Luther King Jr School','May Werthan Shayne Elementary School',
 'Ross Elementary School'  
Stratford STEM Magnet School'
'W.A. Bass Middle School'
 'Cane Ridge High School

In [ ]:
enrollment["SCHOOL_NAME"].unique()

In [ ]:
enrollment["SCHOOL_NAME"] = enrollment["SCHOOL_NAME"].replace('School', '')

In [ ]:
enrollment["SCHOOL_NAME"].unique()

In [ ]:
enrollment['SCHOOL_NAME'] = enrollment['SCHOOL_NAME'].astype(str) + ' School'
enrollment

In [ ]:
if enrollment["SCHOOL_NAME"](-1) == 'School':
        enrollment["SCHOOL_NAME"] == enrollment["SCHOOL_NAME"]
else: 
        

In [ ]:
school_enrollment = pd.merge(left = School_map_data, 
         right = enrollment, 
         on = 'SCHOOL_NAME',
        how = 'inner_join'            
        )

In [ ]:
school_enrollment

Exporting data frame into csv file for use in R 

In [ ]:
School_map_data.to_csv('School_map_data.csv', index = True)

### Part 2
Using the American Community Survey API, obtained for each census tract, the population (B01001_001E in the detailed tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.


In [ ]:
endpoint = 'https://api.census.gov/data/2021/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2021 = requests.get(endpoint)
Population2021 = Pop_response2021.json()
Population2021_df =pd.DataFrame.from_records(Population2021)
Population2021_df

In [ ]:
Population2021_df = Population2021_df.rename(columns = Population2021_df.iloc[0])
Population2021_df = Population2021_df.drop(0)
Population2021_df = Population2021_df.rename(columns = { "B01001_001E" : "2021"})
Population2021_df = Population2021_df.drop(columns = ['state', 'county'])
Population2021_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2020 = requests.get(endpoint)
Population2020 = Pop_response2020.json()
Population2020_df =pd.DataFrame.from_records(Population2020)
Population2020_df

In [ ]:
Population2020_df = Population2020_df.rename(columns = Population2020_df.iloc[0])
Population2020_df = Population2020_df.drop(0)
Population2020_df = Population2020_df.rename(columns = { "B01001_001E" : "2020"})
Population2020_df= Population2020_df.drop(columns = ['state', 'county'])
Population2020_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2019 = requests.get(endpoint)
Population2019 = Pop_response2019.json()
Population2019_df =pd.DataFrame.from_records(Population2019)
Population2019_df

In [ ]:
Population2019_df = Population2019_df.rename(columns = Population2019_df.iloc[0])
Population2019_df = Population2019_df.drop(0)
Population2019_df = Population2019_df.rename(columns = { "B01001_001E" : "2019"})
Population2019_df = Population2019_df.drop(columns = ['state', 'county'])
Population2019_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2018 = requests.get(endpoint)
Population2018 = Pop_response2018.json()
Population2018_df = pd.DataFrame.from_records(Population2018)
Population2018_df

In [ ]:
Population2018_df = Population2018_df.rename(columns = Population2018_df.iloc[0])
Population2018_df = Population2018_df.drop(0)
Population2018_df = Population2018_df.rename(columns = { "B01001_001E" : "2018"})
Population2018_df = Population2018_df.drop(columns = ['state', 'county'])
Population2018_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2017 = requests.get(endpoint)
Population2017 = Pop_response2017.json()
Population2017_df = pd.DataFrame.from_records(Population2017)
Population2017_df

In [ ]:
Population2017_df = Population2017_df.rename(columns = Population2017_df.iloc[0])
Population2017_df = Population2017_df.drop(0)
Population2017_df = Population2017_df.rename(columns = { "B01001_001E" : "2017"})
Population2017_df = Population2017_df.drop(columns = ['state', 'county'])
Population2017_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2016 = requests.get(endpoint)
Population2016 = Pop_response2016.json()
Population2016_df = pd.DataFrame.from_records(Population2016)
Population2016_df

In [ ]:
Population2016_df = Population2016_df.rename(columns = Population2016_df.iloc[0])
Population2016_df = Population2016_df.drop(0)
Population2016_df = Population2016_df.rename(columns = { "B01001_001E" : "2016"})
Population2016_df = Population2016_df.drop(columns = ['state', 'county'])
Population2016_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2015/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2015 = requests.get(endpoint)
Population2015 = Pop_response2015.json()
Population2015_df = pd.DataFrame.from_records(Population2015)
Population2015_df

In [ ]:
Population2015_df = Population2015_df.rename(columns = Population2015_df.iloc[0])
Population2015_df = Population2015_df.drop(0)
Population2015_df = Population2015_df.rename(columns = { "B01001_001E" : "2015"})
Population2015_df = Population2015_df.drop(columns = ['state', 'county'])
Population2015_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2014/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2014 = requests.get(endpoint)
Population2014 = Pop_response2014.json()
Population2014_df = pd.DataFrame.from_records(Population2014)
Population2014_df

In [ ]:
Population2014_df = Population2014_df.rename(columns = Population2014_df.iloc[0])
Population2014_df = Population2014_df.drop(0)
Population2014_df = Population2014_df.rename(columns = { "B01001_001E" : "2014"})
Population2014_df = Population2014_df.drop(columns = ['state', 'county'])
Population2014_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2013/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2013 = requests.get(endpoint)
Population2013 = Pop_response2013.json()
Population2013_df = pd.DataFrame.from_records(Population2013)
Population2013_df

In [ ]:
Population2013_df = Population2013_df.rename(columns = Population2013_df.iloc[0])
Population2013_df = Population2013_df.drop(0)
Population2013_df = Population2013_df.rename(columns = { "B01001_001E" : "2013"})
Population2013_df = Population2013_df.drop(columns = ['state', 'county'])
Population2013_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2012/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2012 = requests.get(endpoint)
Population2012 = Pop_response2012.json()
Population2012_df = pd.DataFrame.from_records(Population2012)
Population2012_df

In [ ]:
Population2012_df = Population2012_df.rename(columns = Population2012_df.iloc[0])
Population2012_df = Population2012_df.drop(0)
Population2012_df = Population2012_df.rename(columns = { "B01001_001E" : "2012"})
Population2012_df = Population2012_df.drop(columns = ['state', 'county'])
Population2012_df 

In [ ]:
endpoint = 'https://api.census.gov/data/2011/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2011 = requests.get(endpoint)
Population2011 = Pop_response2011.json()
Population2011_df = pd.DataFrame.from_records(Population2011)
Population2011_df

In [ ]:
Population2011_df = Population2011_df.rename(columns = Population2011_df.iloc[0])
Population2011_df = Population2011_df.drop(0)
Population2011_df = Population2011_df.rename(columns = { "B01001_001E" : "2011"})
Population2011_df = Population2011_df.drop(columns = ['state', 'county'])
Population2011_df 

In [ ]:
endpoint= 'https://api.census.gov/data/2010/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2010 = requests.get(endpoint)
Population2010 = Pop_response2010.json()
Population2010_df = pd.DataFrame.from_records(Population2010)
Population2010_df

In [ ]:
Population2010_df = Population2010_df.rename(columns = Population2010_df.iloc[0])
Population2010_df = Population2010_df.drop(0)
Population2010_df = Population2010_df.rename(columns = { "B01001_001E" : "2010"})
Population2010_df = Population2010_df.drop(columns = ['state', 'county'])
Population2010_df 

In [ ]:
endpoint= 'https://api.census.gov/data/2009/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47&in=county:037&key=3a2e429d237f872aa3e2e39bc038ee65608e6090'
Pop_response2009 = requests.get(endpoint)
Population2009= Pop_response2009.json()
Population2009_df=pd.DataFrame.from_records(Population2009)
Population2009_df

In [ ]:
Population2009_df = Population2009_df.rename(columns = Population2009_df.iloc[0])
Population2009_df = Population2009_df.drop(0)
Population2009_df = Population2009_df.rename(columns = { "B01001_001E" : "2009"})
Population2009_df = Population2009_df.drop(columns = ['state', 'county'])
Population2009_df 

In [ ]:
Population = pd.merge(left = Population2021_df, 
         right = Population2020_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2019_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2018_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2017_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2016_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2015_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2014_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2013_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2012_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2011_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population = pd.merge(left = Population, 
         right = Population2010_df, 
         left_on = ['NAME', 'tract'], 
         right_on = ['NAME', 'tract'])
Population

In [ ]:
Population.to_csv('population_census_tract.csv', index = True)

In [ ]:
import re